In [ ]:
from solid import cylinder, translate, square, rotate_extrude
from solid.objects import cube, hull, rotate
from stackable_pipe import ring
import math

from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def hook(radius=10, strength=4, angle=60, extruded_length=None):
    out_sq = translate((radius, 0, 0))(square(strength))
    base_sq = translate((radius, 0, 0))(square((0.1, strength)))
    hook_sq = translate((radius+strength, strength-0.1, 0))(square((0.1)))

    if extruded_length is not None:
        angle = math.degrees(math.atan((extruded_length/radius)))

    offset_angle = math.degrees(math.atan((strength/radius)))
    base = rotate_extrude(angle, segments=100)(base_sq)
    hook = rotate((0, 0, offset_angle))(
        rotate_extrude(angle-(offset_angle*2), segments=100)(hook_sq)
    )
    outline = rotate_extrude(angle, segments=100)(out_sq)

    return (rotate(-angle/2)(hull()(base + hook)*outline), rotate(-angle/2)(outline))


def connection(d=80, h=30, angle=20, wall=2, strength=2,hook_angle = 15, gap=0.1):
    radius = d/2
    inner_radius_delta = (h * math.tan(math.radians(angle)))
    inner_radius = radius - inner_radius_delta
    base = ring(r2=radius, r1=inner_radius, h=h, w=wall)

    cut = cube((d, .4, h), center=True)
    (hook_instance, hook_slot) = hook(radius=inner_radius, angle=hook_angle-2)
    cuts = [rotate(z+hook_angle/2)(cut)for z in range(0, 360, hook_angle)]
    hooks = [rotate(z)(hook_instance) for z in range(0, 360, hook_angle*2)]

    adapter = base + hooks - cuts

    h_intercept = h - (wall / math.tan(math.radians(angle)))
    slots = [rotate(z)(hook_slot) for z in range(0, 360, hook_angle*2)]
    socket_base = ring(r2=radius, r1=inner_radius+wall, h=h_intercept, w=wall-gap)
    socket = socket_base - slots

    return (adapter, socket)


(adapter, socket) = connection()

render(adapter, outfile='./outfiles/test_new_adapter.stl')
render(socket, outfile='./outfiles/test_new_socket.stl')
